In [89]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import log
import shutil
import sys
import os.path
#from pyomo.environ import *
import pyomo.environ as pe
import pyomo.opt as po

import logging
# Suppress specific warning
logging.getLogger('pyomo.core').setLevel(logging.ERROR)
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import shutup
shutup.please()

In [90]:
data_b = [36, 35, 38, 34, 32, 34, 31, 34]

data_c = [
    25, 23, 20, 16, 19, 22, 20, 16, 15, 22, 15, 21, 20, 23, 20, 22, 19, 25, 25, 24, 21, 17, 23, 17,
    16, 19, 22, 22, 19, 23, 17, 24, 15, 24, 18, 19, 20, 24, 25, 25, 19, 24, 18, 21, 16, 25, 15, 20,
    20, 18, 23, 23, 23, 17, 19, 16, 24, 24, 17, 23, 19, 22, 23, 25, 23, 18, 19, 24, 20, 17, 23, 23,
    16, 16, 15, 23, 15, 15, 25, 22, 17, 20, 19, 16, 17, 17, 20, 17, 17, 18, 16, 18, 15, 25, 22, 17,
    17, 23, 21, 20, 24, 22, 25, 17, 22, 20, 16, 22, 21, 23, 24, 15, 22, 25, 18, 19, 19, 17, 22, 23,
    24, 21, 23, 17, 21, 19, 19, 17, 18, 24, 15, 15, 17, 18, 15, 24, 19, 21, 23, 24, 17, 20, 16, 21,
    18, 21, 22, 23, 22, 15, 18, 15, 21, 22, 15, 23, 21, 25, 25, 23, 20, 16, 25, 17, 15, 15, 18, 16,
    19, 24, 18, 17, 21, 18, 24, 25, 18, 23, 21, 15, 24, 23, 18, 18, 23, 23, 16, 20, 20, 19, 25, 21
]

data_a = [
    8, 18, 22, 5, 11, 11, 22, 11, 17, 22, 11, 20, 13, 13, 7, 22, 15, 22, 24, 8, 8, 24, 18, 8,
    24, 14, 11, 15, 24, 8, 10, 15, 19, 25, 6, 13, 10, 25, 19, 24, 13, 12, 5, 18, 10, 24, 8, 5,
    22, 22, 21, 22, 13, 16, 21, 5, 25, 13, 12, 9, 24, 6, 22, 24, 11, 21, 11, 14, 12, 10, 20, 6,
    13, 8, 19, 12, 19, 18, 10, 21, 5, 9, 11, 9, 22, 8, 12, 13, 9, 25, 19, 24, 22, 6, 19, 14,
    25, 16, 13, 5, 11, 8, 7, 8, 25, 20, 24, 20, 11, 6, 10, 10, 6, 22, 10, 10, 13, 21, 5, 19,
    19, 19, 5, 11, 22, 24, 18, 11, 6, 13, 24, 24, 22, 6, 22, 5, 14, 6, 16, 11, 6, 8, 18, 10,
    24, 10, 9, 10, 6, 15, 7, 13, 20, 8, 7, 9, 24, 9, 21, 9, 11, 19, 10, 5, 23, 20, 5, 21, 6,
    9, 9, 5, 12, 10, 16, 15, 19, 18, 20, 18, 16, 21, 11, 12, 22, 16, 21, 25, 7, 14, 16, 10
]


### M1 Concrete Model

In [40]:
m1 = pe.ConcreteModel()
# Defining data
#m1.M = pe.RangeSet(1,8)
m1.J = pe.RangeSet(1,24)

In [41]:
# constraints
def kp_rule(m1,i):
    return sum(m1.a[i,j]*m1.x[i,j] for j in m1.J) <= m1.b[i]
#m.kp = pe.Constraint(m.M,rule=kp_rule)    

In [42]:
# FUNCTION TO SOLVE LR Optimal

def solve_lr(i,lam,it):

    global m1
    if it > 1:
        # Attempt to delete components and their index sets
        components_to_delete = ['lda', 'obj', 'x', 'kp', 'c', 'b', 'a']
        for comp_name in components_to_delete:
            # Delete the component if it exists
            if comp_name in m1.component_map():
                m1.del_component(comp_name)
                m1.del_component(f"{comp_name}_index")  # Try to delete the associated index set

    m1.M = pe.RangeSet(1,i)
    
    index = 0
    c = {}
    for i in range(1, i+1):
        for j in range(1, 25):
            c[(i, j)] = data_c[index]
            index += 1 
            
    index = 0
    b = {}      
    b = {i: data_b[i-1] for i in m1.M}

    index = 0
    a = {}
    for i in range(1, i+1):
        for j in range(1, 25):
            a[(i, j)] = data_a[index]
            index += 1
           
    # parameters
    m1.c = pe.Param(m1.M,m1.J, initialize=c)
    m1.b = pe.Param(m1.M, initialize=b)
    m1.a = pe.Param(m1.M,m1.J, initialize=a)
    # variables
    m1.x = pe.Var(m1.M,m1.J, domain=pe.Binary, initialize =0)

    # defining Lambda
    #lmda = {}
    lmda = {j: lam[j-1] for j in m1.J}
    m1.lda  = pe.Param(m1.J, initialize=lmda)
   
    obj_expr = sum(m1.lda[j] for j in m1.J) + sum(sum((m1.c[i,j] - m1.lda[j])*m1.x[i,j] for j in m1.J) for i in  m1.M )
    m1.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)
    m1.kp = pe.Constraint(m1.M,rule=kp_rule) 

    solver = po.SolverFactory('gurobi')
    results = solver.solve(m1,tee=False)
    v = pe.value(m1.obj)
    return v

In [43]:
lam = [10 for jr in m1.J]
v = solve_lr(8,lam,1)
print(v)

673.0


In [44]:
#m1.del_component(m1.c)
lam = [9 for jr in m1.J]
v = solve_lr(5,lam,2)
print(v)

518.0


### M2 Concrete Model

In [91]:
m2 = pe.ConcreteModel()
# Defining data
#m2.M = pe.RangeSet(1,8)
m2.J = pe.RangeSet(1,24)

In [92]:
# Constraints
#1.
def kp_rule(m2,i):
    return sum(m2.a[i,j]*m2.x[i,j] for j in m2.J) <= m2.b[i]

#2. 
def obj_rule(m2):
    return m2.neta >= (sum(m2.lda[j] for j in m2.J) + sum(sum((m2.c[i,j] - m2.lda[j])*m2.x[i,j] for j in m2.J) for i in  m2.M ))

In [93]:
def solve_lp(i,it):

    global m2
    if it > 1:
        # Attempt to delete components and their index sets
        components_to_delete = ['lda', 'obj', 'x', 'kp', 'c', 'b', 'a','obj_cons','neta']
        for comp_name in components_to_delete:
            # Delete the component if it exists
            if comp_name in m2.component_map():
                m2.del_component(comp_name)
                m2.del_component(f"{comp_name}_index")  # Try to delete the associated index set
     
    m2.M = pe.RangeSet(1,i)
    index = 0
    c = {}
    for i in range(1, i+1):
        for j in range(1, 25):
            c[(i, j)] = data_c[index]
            index += 1 
            
    index = 0
    b = {}      
    b = {i: data_b[i-1] for i in m2.M}

    index = 0
    a = {}
    for i in range(1, i+1):
        for j in range(1, 25):
            a[(i, j)] = data_a[index]
            index += 1
           
    # parameters
    m2.c = pe.Param(m2.M,m2.J, initialize=c)
    m2.b = pe.Param(m2.M, initialize=b)
    m2.a = pe.Param(m2.M,m2.J, initialize=a)
    # variables
    m2.x = pe.Var(m2.M,m2.J, domain=pe.Binary)
    m2.neta = pe.Var(domain=pe.Reals)
    m2.lda =  pe.Var(m2.J,domain=pe.NonNegativeReals)
   
    obj_expr = m2.neta
    m2.obj = pe.Objective(sense=pe.minimize, expr=obj_expr)
    m2.kp = pe.Constraint(m2.M,rule=kp_rule) 
    m2.obj_cons = pe.Constraint(rule=obj_rule)

    solver = po.SolverFactory('gurobi')
    results = solver.solve(m2,tee=True)
    neta_opt = pe.value(m2.obj)
    
    return neta_opt

In [94]:
neta_opt = solve_lp(4,1)
print(neta_opt)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25
Read LP format model from file /var/folders/kk/630d4n695dd9k2gbbmxbrcbc0000gp/T/tmpecycmv0f.pyomo.lp
Reading time = 0.00 seconds
x1: 4 rows, 121 columns, 96 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 4 rows, 121 columns and 96 nonzeros
Model fingerprint: 0xf3e9331e
Model has 1 quadratic constraint
Variable types: 25 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 4e+01]
Found heuristic solution: objective 0.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 196 rows, 408 columns, 576 nonzeros

In [88]:
neta_opt = solve_lp(8,3)
print(neta_opt)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-25
Read LP format model from file /var/folders/kk/630d4n695dd9k2gbbmxbrcbc0000gp/T/tmpi2ndqeac.pyomo.lp
Reading time = 0.00 seconds
x1: 8 rows, 217 columns, 192 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.2.0 23C71)

CPU model: Apple M3 Max
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Optimize a model with 8 rows, 217 columns and 192 nonzeros
Model fingerprint: 0x926e545c
Model has 1 quadratic constraint
Variable types: 25 continuous, 192 integer (192 binary)
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+01, 4e+01]
Found heuristic solution: objective 0.0000000
Presolve removed 0 rows and 1 columns
Presolve time: 0.00s
Presolved: 392 rows, 792 columns, 1152 non

In [ ]:
 #### MAIN ENGINE ####

len_b = len(b)
for i in range(1,len(b)+1):

    if( i >1 ):
        # Define Iteration- For the number for which solve_vr is being called.
        call_it = 0
        z_opt = solve_lr(i,lam,call_it)
        call_it += 1



    # cut_it_k = 0
            
    #         for i in m.M:
    
    # if (cut_it_k > 1):
            
    # m.cuts_addition = pe.ConstraintList()
    # lhs = sum(m.a[i,j]*m.x[i,j] for  j in m.M)
    # rhs = m.b[i]



    


In [4]:
# OLD

# m = pe.ConcreteModel()
# # Defining data
# m.M = pe.RangeSet(1,8)
# m.J = pe.RangeSet(1,24)
# data_b = [36, 35, 38, 34, 32, 34, 31, 34]
# # Create a dictionary with keys from 1 to 8 and corresponding values from list b
# b = {i: data_b[i-1] for i in m.M}
# ##################

# data_c = [
#     25, 23, 20, 16, 19, 22, 20, 16, 15, 22, 15, 21, 20, 23, 20, 22, 19, 25, 25, 24, 21, 17, 23, 17,
#     16, 19, 22, 22, 19, 23, 17, 24, 15, 24, 18, 19, 20, 24, 25, 25, 19, 24, 18, 21, 16, 25, 15, 20,
#     20, 18, 23, 23, 23, 17, 19, 16, 24, 24, 17, 23, 19, 22, 23, 25, 23, 18, 19, 24, 20, 17, 23, 23,
#     16, 16, 15, 23, 15, 15, 25, 22, 17, 20, 19, 16, 17, 17, 20, 17, 17, 18, 16, 18, 15, 25, 22, 17,
#     17, 23, 21, 20, 24, 22, 25, 17, 22, 20, 16, 22, 21, 23, 24, 15, 22, 25, 18, 19, 19, 17, 22, 23,
#     24, 21, 23, 17, 21, 19, 19, 17, 18, 24, 15, 15, 17, 18, 15, 24, 19, 21, 23, 24, 17, 20, 16, 21,
#     18, 21, 22, 23, 22, 15, 18, 15, 21, 22, 15, 23, 21, 25, 25, 23, 20, 16, 25, 17, 15, 15, 18, 16,
#     19, 24, 18, 17, 21, 18, 24, 25, 18, 23, 21, 15, 24, 23, 18, 18, 23, 23, 16, 20, 20, 19, 25, 21
# ]

# # Define the dictionary
# c = {}

# # Populate the dictionary
# index = 0
# for i in range(1, 9):
#     for j in range(1, 25):
#         c[(i, j)] = data_c[index]
#         index += 1  
# ##################

# data_a = [
#     8, 18, 22, 5, 11, 11, 22, 11, 17, 22, 11, 20, 13, 13, 7, 22, 15, 22, 24, 8, 8, 24, 18, 8,
#     24, 14, 11, 15, 24, 8, 10, 15, 19, 25, 6, 13, 10, 25, 19, 24, 13, 12, 5, 18, 10, 24, 8, 5,
#     22, 22, 21, 22, 13, 16, 21, 5, 25, 13, 12, 9, 24, 6, 22, 24, 11, 21, 11, 14, 12, 10, 20, 6,
#     13, 8, 19, 12, 19, 18, 10, 21, 5, 9, 11, 9, 22, 8, 12, 13, 9, 25, 19, 24, 22, 6, 19, 14,
#     25, 16, 13, 5, 11, 8, 7, 8, 25, 20, 24, 20, 11, 6, 10, 10, 6, 22, 10, 10, 13, 21, 5, 19,
#     19, 19, 5, 11, 22, 24, 18, 11, 6, 13, 24, 24, 22, 6, 22, 5, 14, 6, 16, 11, 6, 8, 18, 10,
#     24, 10, 9, 10, 6, 15, 7, 13, 20, 8, 7, 9, 24, 9, 21, 9, 11, 19, 10, 5, 23, 20, 5, 21, 6,
#     9, 9, 5, 12, 10, 16, 15, 19, 18, 20, 18, 16, 21, 11, 12, 22, 16, 21, 25, 7, 14, 16, 10
# ]
# # Define the dictionary
# a = {}
# # Populate the dictionary
# index = 0
# for i in range(1, 9):
#     for j in range(1, 25):
#         a[(i, j)] = data_a[index]
#         index += 1

# # Initialising
# m.b = pe.Param(m.M, initialize=b)
# m.c = pe.Param(m.M,m.J, initialize=c)
# m.a = pe.Param(m.M,m.J, initialize=a)


# # Defining variable
# m.x = pe.Var(m.M,m.J, domain=pe.Binary, initialzie = 0)
# m.lamb = pe.Var(domain=pe.NonNegativeReals)
# m.neta = pe.Var(domain=pe.Reals)


# # def ObjRule(model):
# #     return 2*model.x[1] + 3*model.x[2]
# # model.obj1 = pyo.Objective(rule=ObjRule)
# def objective_rule(m):
#     val = sum(m.lda[j] for j in m.J) + sum(sum((m.c[i,j] - m.lda[j])*m.x[i,j] for j in m.J) for i in  m.M )
#     return val
# #m.obj = pe.Objective(rule=objective_rule,sense=pe.maximize)

In [193]:
# this function gives optimum value of x for a particular lambda_k
def opt_x(m,lam):
    #lam = {j: l[j-1] for j in m.J}
    #lam = ltodict(l)
    m.lda = pe.Param(m.J, initialize=lam)
    m.obj = pe.Objective(rule=objective_rule,sense=pe.maximize)
    m.kp = pe.Constraint(m.M,rule=kp_rule) 
    solver = po.SolverFactory('gurobi')
    results = solver.solve(m,tee=False)
    #print(pe.value(m.obj))


opt_x(m,lam)

# Now, we have x^k for a particular lambda^k

# Further, Now
# At this point, one could check how many (out of j) of MC constraints does the problem NOT satisfy

# If the not satisfying answer is zero, this means: The Lagrg Solution is feasible for Primal.
def mc_const_count(m):
    it =0
    for j in m.J:
        if((sum(pe.value(m.x[i,j]) for i in m.M)) ==1):
            it = it+1
    it = 24 - it
    return it


cmc = mc_const_count(m)
print("No. of MC constraints (out of 24) not satisfied =",cmc,'\n Thus, Langrangian solution is not feasible for Primal.')


# Implementing Sub-gradient Optimisation 

                ##### METHOD 1 #####
# 1. Using known value of Optimal Solution i.e. Neta^*

                ##### METHOD 2 ##### 
# 2. Intially Neta^* = 0 (set as 0)- then update the value on those iteartions in which the lagrangian problem 
# turns out to be feasible in the original problem (Primal Problem)


                #### METHOD 1 #####

opt = 563
neta = 673
eps = 1e-3
it =0
while (it < 1000):
    arrl = np.array(l)
    s=[]                  # Finding s^k as List
    for j in m.J:
        v = 1 - sum(pe.value(m.x[i,j]) for i in m.M)
        s.append(v)
    arrs = np.array(s)    # Converting s^k to Numpy Array
    norms = np.linalg.norm(arrs)**2
    addterm = arrs*(opt-neta)/norms
    ln = arrl + addterm
    ln = ln.tolist()
    l = ln
    lam = {j: l[j-1] for j in m.J}
    # Update Objective and x-values
    opt_x(m,lam)
    neta = pe.value(m.obj)
    it = it+1
